In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot

In [2]:
data = pd.read_csv('../data/US_Accidents_Dec19.csv')

In [3]:
data.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [4]:
data.columns

Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [14]:
corr_df = data[['ID', 'Severity', 'City', 'State', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset']].copy()

In [11]:
corr_df.head()

,ID,Severity,City,State,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),...,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset
0,A-1,3,Dayton,OH,36.9,NaN,91.0,29.68,10.0,NaN,...,False,False,False,False,False,False,False,False,False,Night
1,A-2,2,Reynoldsburg,OH,37.9,NaN,100.0,29.65,10.0,NaN,...,False,False,False,False,False,False,False,False,False,Night
2,A-3,2,Williamsburg,OH,36.0,33.3,100.0,29.67,10.0,3.5,...,False,False,False,False,False,False,False,True,False,Night
3,A-4,3,Dayton,OH,35.1,31.0,96.0,29.64,9.0,4.6,...,False,False,False,False,False,False,False,False,False,Night
4,A-5,2,Dayton,OH,36.0,33.3,89.0,29.65,6.0,3.5,...,False,False,False,False,False,False,False,True,False,Day


In [15]:
night_dct = {'Night':True, 'Day':False}

In [17]:
corr_df['Night'] = corr_df['Sunrise_Sunset'].map(night_dct)

In [20]:
corr_df.drop(columns='Sunrise_Sunset', inplace=True)

In [21]:
corr_df.head()

,ID,Severity,City,State,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),...,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Traffic_Calming,Traffic_Signal,Turning_Loop,Night
0,A-1,3,Dayton,OH,36.9,NaN,91.0,29.68,10.0,NaN,...,False,False,False,False,False,False,False,False,False,True
1,A-2,2,Reynoldsburg,OH,37.9,NaN,100.0,29.65,10.0,NaN,...,False,False,False,False,False,False,False,False,False,True
2,A-3,2,Williamsburg,OH,36.0,33.3,100.0,29.67,10.0,3.5,...,False,False,False,False,False,False,False,True,False,True
3,A-4,3,Dayton,OH,35.1,31.0,96.0,29.64,9.0,4.6,...,False,False,False,False,False,False,False,False,False,True
4,A-5,2,Dayton,OH,36.0,33.3,89.0,29.65,6.0,3.5,...,False,False,False,False,False,False,False,True,False,False


In [22]:
corr_df.corr()

,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Traffic_Calming,Traffic_Signal,Turning_Loop
Severity,1.000000,-0.029132,-0.079993,0.023188,0.018289,-0.006136,0.030343,0.016632,-0.005455,-0.129030,-0.007847,0.063437,-0.006709,-0.023666,-0.003843,-0.006786,-0.195834,NaN
Temperature(F),-0.029132,1.000000,0.994869,-0.327122,-0.072190,0.175596,-0.029893,-0.013120,-0.000087,0.028324,-0.000564,-0.007453,0.003662,0.005254,-0.001117,-0.001611,0.053912,NaN
Wind_Chill(F),-0.079993,0.994869,1.000000,-0.308035,-0.235248,0.190564,-0.192466,-0.036659,0.002761,0.029448,-0.000019,-0.015242,0.004638,0.014252,-0.001561,0.001751,0.040450,NaN
Humidity(%),0.023188,-0.327122,-0.308035,1.000000,0.091434,-0.377845,-0.133499,0.061062,-0.001886,-0.031930,0.003124,-0.006833,-0.008213,-0.001758,0.001553,0.000339,-0.019388,NaN
Pressure(in),0.018289,-0.072190,-0.235248,0.091434,1.000000,-0.020050,0.017755,0.039764,-0.005220,-0.012027,0.001191,0.026430,-0.006367,0.008985,0.001737,-0.000685,0.006766,NaN
Visibility(mi),-0.006136,0.175596,0.190564,-0.377845,-0.020050,1.000000,0.007981,-0.083210,0.003847,0.031827,0.004690,0.000221,0.012004,0.002663,0.000036,0.002633,0.025041,NaN
Wind_Speed(mph),0.030343,-0.029893,-0.192466,-0.133499,0.017755,0.007981,1.000000,0.026155,0.000233,0.005511,-0.001977,0.010770,0.000688,-0.005383,-0.000951,-0.002531,0.010205,NaN
Precipitation(in),0.016632,-0.013120,-0.036659,0.061062,0.039764,-0.083210,0.026155,1.000000,-0.000743,0.001136,-0.001330,0.008951,-0.001229,0.001535,-0.000192,-0.000964,0.008412,NaN
Bump,-0.005455,-0.000087,0.002761,-0.001886,-0.005220,0.003847,0.000233,-0.000743,1.000000,0.010501,-0.000626,-0.003046,0.022150,-0.000250,-0.000093,0.639180,-0.003545,NaN
Crossing,-0.129030,0.028324,0.029448,-0.031930,-0.012027,0.031827,0.005511,0.001136,0.010501,1.000000,0.072498,-0.074802,0.053187,0.242318,-0.000830,0.023317,0.433672,NaN
